In [1]:
import MDAnalysis
import MDAnalysis.analysis.rdf
import MDAnalysis.analysis.msd

/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
MDAnalysis.Universe("../Data/equil/kalj_T1.5_n360_v300_10000_1.lammpstrj",
                    "../Data/prod/kalj_T1.5_n360_v300_prod_10000_1.unwrap.dcd",
                    format="LAMMPSDUMP")

ValueError: 'LAMMPSTRJ' isn't a valid topology format, nor a coordinate format
   from which a topology can be minimally inferred.
   You can use 'Universe(topology, ..., topology_format=FORMAT)'
   to explicitly specify the format and
   override automatic detection. Known FORMATs are:
   dict_keys(['PSF', 'TOP', 'PRMTOP', 'PARM7', 'PDB', 'ENT', 'XPDB', 'PQR', 'GRO', 'CRD', 'PDBQT', 'DMS', 'TPR', 'MOL2', 'DATA', 'LAMMPSDUMP', 'XYZ', 'TXYZ', 'ARC', 'GMS', 'CONFIG', 'HISTORY', 'XML', 'MMTF', 'GSD', 'MINIMAL', 'ITP', 'IN', 'FHIAIMS', 'PARMED', 'RDKIT', 'OPENMMTOPOLOGY', 'OPENMMAPP'])
   See https://docs.mdanalysis.org/documentation_pages/topology/init.html#supported-topology-formats
   For missing formats, raise an issue at 
   https://github.com/MDAnalysis/mdanalysis/issues

In [2]:
run = {'1.5': '10000', '1': '50000',
       '0.9': '150000', '0.8': '150000', 
       '0.7': '180000', '0.65': '350000', 
       '0.6': '1000000', '0.55': '3000000', 
       '0.5': '10000000', '0.475': '30000000', 
       '0.45': '100000000'}
equi_data = dict(zip(run.keys(), [None]*len(run.keys())))
prod_data = dict(zip(run.keys(), [None]*len(run.keys())))
for t in [1.5, 1, 0.9, 0.8, 0.7, 0.65, 0.6, 0.55, 0.5, 0.475]:
    equi_data[str(t)] = MDAnalysis.Universe(f"../Data/equil/kalj_T{t}_n360_v300_{run[str(t)]}_1.lammpstrj",
                    format="LAMMPSDUMP")
    prod_data[str(t)] = MDAnalysis.Universe(f"../Data/prod/kalj_T{t}_n360_v300_prod_{run[str(t)]}_1.unwrap.dcd")

/ext3/miniconda3/lib/python3.9/site-packages/MDAnalysis/topology/LAMMPSParser.py:631: UserWarning: Guessed all Masses to 1.0
  warnings.warn('Guessed all Masses to 1.0')


In [24]:
def cal_rdf(prod_data):
    type1 = prod_data.select_atoms("type 1")
    type2 = prod_data.select_atoms("type 2")

    rdf_bins = 200
    rdf_range = (0.0, 15.0)

    rdf_11 = rdf.InterRDF(type_1, type_1, nbins=rdf_bins, range=rdf_range).run()
    rdf_22 = rdf.InterRDF(type_2, type_2, nbins=rdf_bins, range=rdf_range).run()
    rdf_12 = rdf.InterRDF(type_1, type_2, nbins=rdf_bins, range=rdf_range).run()
    return rdf_11, rdf_22, rdf_12

In [27]:
cal_rdf(prod_data['1.5'])

AttributeError: 'Topology' object has no attribute 'types'

In [38]:
prod_data['1.5'].atoms.describe()

AttributeError: AtomGroup has no attribute describe. 

In [4]:
def cal_rdf_msd(lamp_path, dvd_path):
    # Set up the Universe
    u = MDAnalysis.Universe(lamp_path, dvd_path, format="LAMMPS")
    # Select atoms for RDF calculation
    rdf_group = u.select_atoms("all")
    # Set up the RDF analysis
    # The range of distances (r_range) and the number of bins (bins) can be adjusted as needed
    rdf = MDAnalysis.analysis.rdf.InterRDF(rdf_group, rdf_group, nbins=150, range=(0.0, 15.0))
    # Run the RDF analysis
    rdf.run()
    # For the MSD, select the atom group you are interested in
    # Here we take all atoms, but you can choose a specific group
    msd_group = u.select_atoms("all")
    # Set up the MSD analysis
    # You can specify the 'start', 'stop', and 'step' arguments as needed
    msd = MDAnalysis.analysis.msd.EinsteinMSD(msd_group, msd_selection='all')
    # Run the MSD analysis
    msd.run()
return rdf.bins, rdf.rdf, msd.times, msd.msds

# Now, plot the RDF
import matplotlib.pyplot as plt

plt.plot(rdf.bins, rdf.rdf)
plt.xlabel('Distance (angstrom)')
plt.ylabel('RDF')
plt.title('Radial Distribution Function')
plt.savefig('rdf_plot.png')
plt.show()


# Now, plot the MSD
plt.plot(msd.times, msd.msds)
plt.xlabel('Time (ps)')
plt.ylabel('MSD (angstrom^2)')
plt.title('Mean Squared Displacement')
plt.savefig('msd_plot.png')
plt.show()


/ext3/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
